# Import necessary packages

In [1]:
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd

import seaborn as sns
import matplotlib.pyplot as plt

from random import sample

import re

from sklearn.metrics import mean_absolute_error, mean_squared_error

from timeit import default_timer as timer

# Import original data and preprocessed data

In [2]:
INPUT_DIR = r'C:\Users\ASUS\Desktop\Academic Work\Academic Research\DEMAND PATTERN\M5 Forecasting\Estimate the unit sales of Walmart retail goods\m5-forecasting-accuracy_unzip files'
sales_train_val = pd.read_csv(f'{INPUT_DIR}/sales_train_validation.csv')
calendar = pd.read_csv(f'{INPUT_DIR}/calendar.csv')

### Note: This Step is for the extraction of California State and Hobbies category

In [3]:
sales_train_val = sales_train_val[(sales_train_val['state_id'] == 'CA') & (sales_train_val['cat_id'] == 'HOBBIES')]

In [4]:
INPUT_DIR_2 = r'C:\Users\ASUS\Desktop\Academic Work\Academic Research\DEMAND PATTERN\CRUNCH TIME (WEEKLY AGGREGATION)\California\Hobbies\Demand Pattern Categorization'
list_intermittent = pd.read_csv(f'{INPUT_DIR_2}/Intermittent_ID.csv')
list_lumpy = pd.read_csv(f'{INPUT_DIR_2}/Lumpy_ID.csv')
list_erratic = pd.read_csv(f'{INPUT_DIR_2}/Erratic_ID.csv')
list_smooth = pd.read_csv(f'{INPUT_DIR_2}/Smooth_ID.csv')

list_intermittent = list_intermittent['0'].values.tolist()
list_lumpy = list_lumpy['0'].values.tolist()
list_erratic = list_erratic['0'].values.tolist()
list_smooth = list_smooth['0'].values.tolist()

sales_intermittent = sales_train_val[sales_train_val.id.isin(list_intermittent)]
sales_lumpy = sales_train_val[sales_train_val.id.isin(list_lumpy)]
sales_erratic = sales_train_val[sales_train_val.id.isin(list_erratic)]
sales_smooth = sales_train_val[sales_train_val.id.isin(list_smooth)]

**Check by looking at column "id" to verify if we choose the right product (FOODS or HOBBIES or HOUSEHOLD)**

In [5]:
sales_lumpy

,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,...,d_1904,d_1905,d_1906,d_1907,d_1908,d_1909,d_1910,d_1911,d_1912,d_1913
64,HOBBIES_1_066_CA_1_validation,HOBBIES_1_066,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,4,0,0,0,0,0,0,6,0
126,HOBBIES_1_131_CA_1_validation,HOBBIES_1_131,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,2,0,0,0,1,0,0,1,0
135,HOBBIES_1_140_CA_1_validation,HOBBIES_1_140,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,4,0,0,0,0,0,1,0,4
140,HOBBIES_1_145_CA_1_validation,HOBBIES_1_145,HOBBIES_1,HOBBIES,CA_1,CA,0,12,0,4,...,0,11,0,0,1,3,0,12,0,2
162,HOBBIES_1_167_CA_1_validation,HOBBIES_1_167,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,0,0,0,0,0,0,0,4,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9703,HOBBIES_2_141_CA_4_validation,HOBBIES_2_141,HOBBIES_2,HOBBIES,CA_4,CA,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
9704,HOBBIES_2_142_CA_4_validation,HOBBIES_2_142,HOBBIES_2,HOBBIES,CA_4,CA,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9709,HOBBIES_2_147_CA_4_validation,HOBBIES_2_147,HOBBIES_2,HOBBIES,CA_4,CA,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
9710,HOBBIES_2_148_CA_4_validation,HOBBIES_2_148,HOBBIES_2,HOBBIES,CA_4,CA,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


# User-defined functions to calculate Metrics and Algorithm

In [6]:
ROUNDING_DECIMAL = 4

def mase_calculation(ts, prediction):
    divisor = 0
    for i in range(1, ts.shape[0]):
        divisor = divisor + abs(ts.iloc[i] - ts.iloc[i-1])
    divisor = divisor/(ts.shape[0] - 1)
    diff    = abs(ts - prediction[:ts.shape[0]])/divisor
    mase    = diff.mean()
    return mase

def mape_calculation(actual, pred): 
    if not all([isinstance(actual, np.ndarray), isinstance(pred, np.ndarray)]):
        actual, pred = np.array(actual), np.array(pred)
    mask = (actual != 0)
    return round((np.fabs(actual - pred)/actual)[mask].mean()*100, ROUNDING_DECIMAL)

def wmape_calculation(actual, pred):
    if not all([isinstance(actual, np.ndarray), isinstance(pred, np.ndarray)]):
        actual, pred = np.array(actual), np.array(pred)
    return round((np.sum(np.absolute(actual-pred))/np.sum(actual))*100, ROUNDING_DECIMAL)

def smape_calculation(actual, predicted):
    if not all([isinstance(actual, np.ndarray), isinstance(predicted, np.ndarray)]):
        actual, predicted = np.array(actual), np.array(predicted)
    return round(np.mean(np.abs(predicted - actual) / ((np.abs(predicted) + np.abs(actual))/2))*100, ROUNDING_DECIMAL)

In [7]:
def Croston_TSB(ts,extra_periods=1,alpha=0.4,beta=0.4):
    d = np.array(ts) # Transform the input into a numpy array
    cols = len(d) # Historical period length
    d = np.append(d,[np.nan]*extra_periods) # Append np.nan into the demand array to cover future periods
    
    #level (a), probability(p) and forecast (f)
    a,p,f = np.full((3,cols+extra_periods),np.nan)
# Initialization
    first_occurence = np.argmax(d[:cols]>0)
    a[0] = d[first_occurence]
    p[0] = 1/(1 + first_occurence)
    f[0] = p[0]*a[0]
                 
    # Create all the t+1 forecasts
    for t in range(0,cols): 
        if d[t] > 0:
            a[t+1] = alpha*d[t] + (1-alpha)*a[t] 
            p[t+1] = beta*(1) + (1-beta)*p[t]  
        else:
            a[t+1] = a[t]
            p[t+1] = (1-beta)*p[t]       
        f[t+1] = p[t+1]*a[t+1]
        
    # Future Forecast
    a[cols+1:cols+extra_periods] = a[cols]
    p[cols+1:cols+extra_periods] = p[cols]
    f[cols+1:cols+extra_periods] = f[cols]
                      
    df = pd.DataFrame.from_dict({"Demand":d,"Forecast":f,"Period":p,"Level":a,"Error":d-f})
    return df

# Format sales data and then merge with calendar data

In [8]:
sales_pattern = sales_lumpy.copy()

In [9]:
sales_pattern_py = sales_pattern.copy()

list_pattern_py = sales_pattern_py.id.unique().tolist()
list_pattern_py = [(re.sub(r'_validation', r"", x)) for x in list_pattern_py]

sales_pattern_py = sales_pattern_py.drop(['item_id', 'dept_id', 'cat_id', 'store_id', 'state_id'], axis=1)
df_pattern = sales_pattern_py.melt(['id'], var_name='Date').pivot(index = ['Date'], columns = 'id', values = 'value').reset_index()
df_pattern.drop('Date', axis=1, inplace=True)
df_pattern.index = pd.date_range('2011-01-29', periods=df_pattern.shape[0], freq="D")
df_pattern.index.names = ['Date']
df_pattern = df_pattern.astype('float64')

df_pattern.iloc[:, 1:].columns = df_pattern.iloc[:, 1:].columns.str.rstrip('_validation')
df_pattern.columns = df_pattern.columns.str.replace(r'_validation', '')
df_pattern.reset_index(inplace=True)

calendar['Date'] = pd.to_datetime(calendar['date'])

dataframe = pd.merge(df_pattern,
                     calendar[['wm_yr_wk', 'Date']],
                     on = 'Date')

not_enough_seven_days = dataframe.groupby('wm_yr_wk').agg({'Date': 'count'})[dataframe.groupby('wm_yr_wk').agg({'Date': 'count'})['Date'] < 7].index.tolist()
dataframe = dataframe[~dataframe['wm_yr_wk'].isin(not_enough_seven_days)]

dataframe.set_index('wm_yr_wk', inplace=True)

dataframe.drop('Date', axis=1, inplace=True)

dataframe_weekly = dataframe.groupby('wm_yr_wk').sum().reset_index()

In [10]:
start_train_week = 11318
end_train_week = 11610
n_pred_weeks = 2
num_train_needed = 100
validation_training_ratio = 0.95
num_id_sample_for_tuning = 200

In [11]:
list_params_alpha = [round(item, 2) for item in list(np.arange(0.05, 0.95, 0.05))]
list_params_beta = [round(item, 2) for item in list(np.arange(0.05, 0.95, 0.05))]

In [15]:
len(list_pattern_py)

395

In [16]:
sample_list = sample(list_pattern_py, num_id_sample_for_tuning)

dataframe_check = dataframe_weekly[sample_list].iloc[-10:-2, :].T
dataframe_check['Check'] = np.where((dataframe_check==0).all(axis=1), 1, 0)

sample_removed = dataframe_check[dataframe_check['Check'] == 1].index.tolist()
sample_list_filtered = [x for x in sample_list if (x not in sample_removed)]

In [17]:
len(sample_list_filtered)

134

# Hyperparameter tuning for Croston TSB

In [12]:
train_data = dataframe_weekly[(dataframe_weekly['wm_yr_wk'] >= start_train_week) & (dataframe_weekly['wm_yr_wk'] <= end_train_week)]
test_data = dataframe_weekly.iloc[n_pred_weeks*-1:]

train_data_first_part = train_data.iloc[:int(len(train_data)*validation_training_ratio), :]
train_data_second_part = train_data.iloc[int(len(train_data)*validation_training_ratio):, :]

In [19]:
start = timer()

df_result_all_params = pd.DataFrame()

for product in sample_list_filtered:
    
    print('Currently Running: %s' % product)
    print('Progressing: {0} %'.format(round(sample_list_filtered.index(product) / len(sample_list_filtered) * 100, 2)))

    train_first = train_data_first_part[product]
    train_second = train_data_second_part[product]
    
    for i in list_params_alpha:
        for j in list_params_beta:
            predictions = list()
            history = [x for x in train_first[num_train_needed*-1:]]
            for t in train_second.index.tolist():
                yhat = Croston_TSB(history, extra_periods=1, alpha = i, beta = j)['Forecast'].iloc[-1]
                predictions.append(yhat)
                history.append(train_second[t])
            df_result_temp = pd.DataFrame({'Product': [product for count in range(len(train_second))],
                                           'Actual Data': train_second,
                                           'Forecast': predictions,
                                           'Alpha': [i for count_i in range(len(train_second))],
                                           'Beta': [j for count_j in range(len(train_second))]})
            df_result_all_params = df_result_all_params.append(df_result_temp, ignore_index=True)    
            
end = timer()
print('This line of code took {} minutes'.format((end-start) / 60))

Currently Running: FOODS_3_470_CA_4
Progressing: 0.0 %
Currently Running: FOODS_3_379_CA_3
Progressing: 0.75 %
Currently Running: FOODS_1_113_CA_3
Progressing: 1.49 %
Currently Running: FOODS_3_240_CA_4
Progressing: 2.24 %
Currently Running: FOODS_3_093_CA_1
Progressing: 2.99 %
Currently Running: FOODS_3_204_CA_2
Progressing: 3.73 %
Currently Running: FOODS_3_463_CA_4
Progressing: 4.48 %
Currently Running: FOODS_2_336_CA_3
Progressing: 5.22 %
Currently Running: FOODS_3_772_CA_2
Progressing: 5.97 %
Currently Running: FOODS_2_169_CA_1
Progressing: 6.72 %
Currently Running: FOODS_3_779_CA_2
Progressing: 7.46 %
Currently Running: FOODS_3_120_CA_4
Progressing: 8.21 %
Currently Running: FOODS_3_611_CA_2
Progressing: 8.96 %
Currently Running: FOODS_3_117_CA_1
Progressing: 9.7 %
Currently Running: FOODS_3_345_CA_3
Progressing: 10.45 %
Currently Running: FOODS_3_027_CA_3
Progressing: 11.19 %
Currently Running: FOODS_1_028_CA_1
Progressing: 11.94 %
Currently Running: FOODS_2_380_CA_1
Progressing

In [20]:
df_result_all_params

,Product,Actual Data,Forecast,Alpha,Beta
0,FOODS_3_470_CA_4,3.0,4.168800,0.05,0.05
1,FOODS_3_470_CA_4,5.0,4.127194,0.05,0.05
2,FOODS_3_470_CA_4,4.0,4.170375,0.05,0.05
3,FOODS_3_470_CA_4,9.0,4.168876,0.05,0.05
4,FOODS_3_470_CA_4,3.0,4.381454,0.05,0.05
...,...,...,...,...,...
347323,FOODS_3_772_CA_4,1.0,1.104869,0.90,0.90
347324,FOODS_3_772_CA_4,0.0,1.010487,0.90,0.90
347325,FOODS_3_772_CA_4,0.0,0.101049,0.90,0.90
347326,FOODS_3_772_CA_4,0.0,0.010105,0.90,0.90


# Summay all metrics

In [13]:
def get_metrics_result_all_params(data):
    metrics_df = {}
    metrics_df['MASE'] = mase_calculation(data['Actual Data'], data['Forecast'])
    metrics_df['WMAPE'] = wmape_calculation(data['Actual Data'], data['Forecast'])
    metrics_df['SMAPE'] = smape_calculation(data['Actual Data'], data['Forecast'])
    metrics_df['MAPE'] = mape_calculation(data['Actual Data'], data['Forecast'])
    metrics_df['MAE'] = mean_absolute_error(data['Actual Data'], data['Forecast'])
    metrics_df['RMSE'] = np.sqrt(mean_squared_error(data['Actual Data'], data['Forecast']))
    return pd.Series(metrics_df)

In [22]:
start = timer()

df_result_metrics_all_params = df_result_all_params.groupby(['Product', 'Alpha', 'Beta']).apply(get_metrics_result_all_params).reset_index()

end = timer()
print('This line of code took {} minutes'.format((end-start) / 60))

This line of code took 1.2488908399999994 minutes


In [23]:
df_result_metrics_all_params

,Product,Alpha,Beta,MASE,WMAPE,SMAPE,MAPE,MAE,RMSE
0,FOODS_1_010_CA_4,0.05,0.05,0.671487,61.3931,77.4099,35.8254,1.151120,1.391805
1,FOODS_1_010_CA_4,0.05,0.10,0.672925,61.5245,78.9319,34.1481,1.153585,1.373856
2,FOODS_1_010_CA_4,0.05,0.15,0.657064,60.0744,78.3213,30.8207,1.126395,1.362207
3,FOODS_1_010_CA_4,0.05,0.20,0.631725,57.7577,75.8777,26.8948,1.082957,1.351727
4,FOODS_1_010_CA_4,0.05,0.25,0.634021,57.9677,77.8316,29.5644,1.086894,1.340357
...,...,...,...,...,...,...,...,...,...
43411,FOODS_3_825_CA_2,0.90,0.70,1.074091,32.4254,34.1985,36.9541,2.148182,2.673688
43412,FOODS_3_825_CA_2,0.90,0.75,1.074090,32.4254,34.1984,36.9541,2.148181,2.673685
43413,FOODS_3_825_CA_2,0.90,0.80,1.074090,32.4254,34.1984,36.9541,2.148181,2.673684
43414,FOODS_3_825_CA_2,0.90,0.85,1.074090,32.4254,34.1984,36.9541,2.148181,2.673684


# Check what products has unexpected metrics result

In [24]:
df_result_metrics_all_params[df_result_metrics_all_params['MASE'].isin([np.inf, -np.inf, np.nan])]

,Product,Alpha,Beta,MASE,WMAPE,SMAPE,MAPE,MAE,RMSE


In [25]:
df_result_metrics_all_params[df_result_metrics_all_params['WMAPE'].isin([np.inf, -np.inf, np.nan])]

,Product,Alpha,Beta,MASE,WMAPE,SMAPE,MAPE,MAE,RMSE


In [26]:
df_result_metrics_all_params[df_result_metrics_all_params['SMAPE'].isin([np.inf, -np.inf, np.nan])]

,Product,Alpha,Beta,MASE,WMAPE,SMAPE,MAPE,MAE,RMSE
12960,FOODS_2_125_CA_4,0.05,0.05,0.480191,109.7580,NaN,100.0000,0.548790,1.417578
12961,FOODS_2_125_CA_4,0.05,0.10,0.520625,119.0000,NaN,100.0000,0.595000,1.426955
12962,FOODS_2_125_CA_4,0.05,0.15,0.558906,127.7500,NaN,100.0000,0.638750,1.441358
12963,FOODS_2_125_CA_4,0.05,0.20,0.595000,136.0000,NaN,100.0000,0.680000,1.459863
12964,FOODS_2_125_CA_4,0.05,0.25,0.628906,143.7500,NaN,100.0000,0.718750,1.481659
...,...,...,...,...,...,...,...,...,...
42115,FOODS_3_806_CA_1,0.90,0.70,1.003044,51.6975,NaN,64.2476,3.295718,5.068135
42116,FOODS_3_806_CA_1,0.90,0.75,0.980773,50.5496,NaN,63.2374,3.222539,4.965833
42117,FOODS_3_806_CA_1,0.90,0.80,0.962384,49.6019,NaN,62.3738,3.162120,4.882464
42118,FOODS_3_806_CA_1,0.90,0.85,0.947491,48.8343,NaN,61.6457,3.113183,4.815187


In [27]:
df_result_metrics_all_params[df_result_metrics_all_params['MAPE'].isin([np.inf, -np.inf, np.nan])]

,Product,Alpha,Beta,MASE,WMAPE,SMAPE,MAPE,MAE,RMSE


In [28]:
df_result_metrics_all_params[df_result_metrics_all_params['RMSE'].isin([np.inf, -np.inf, np.nan])]

,Product,Alpha,Beta,MASE,WMAPE,SMAPE,MAPE,MAE,RMSE


In [29]:
df_result_metrics_all_params[df_result_metrics_all_params['MAE'].isin([np.inf, -np.inf, np.nan])]

,Product,Alpha,Beta,MASE,WMAPE,SMAPE,MAPE,MAE,RMSE


# Metrics Statistics

In [30]:
list_metrics = ['MASE', 'WMAPE', 'SMAPE', 'MAPE', 'MAE', 'RMSE']

In [31]:
for mts in list_metrics:
    print('Percentage of unexpected values of', mts, 'is: {}'.format(df_result_metrics_all_params[df_result_metrics_all_params[mts].isin([np.nan, np.inf, -np.inf])].Product.nunique() / df_result_metrics_all_params.Product.nunique() * 100), "%")

Percentage of unexpected values of MASE is: 0.0 %
Percentage of unexpected values of WMAPE is: 0.0 %
Percentage of unexpected values of SMAPE is: 2.9850746268656714 %
Percentage of unexpected values of MAPE is: 0.0 %
Percentage of unexpected values of MAE is: 0.0 %
Percentage of unexpected values of RMSE is: 0.0 %


### Filter all rows that have unexpected metrics values

In [32]:
df_result_metrics_all_params = df_result_metrics_all_params[~df_result_metrics_all_params.isin([np.nan, np.inf, -np.inf]).any(1)]

# Get MEAN metrics value of each Alpha and Beta

In [33]:
df_result_metrics_all_params.groupby(['Alpha', 'Beta'])['MASE', 'WMAPE', 'SMAPE', 'MAPE', 'MAE', 'RMSE'].mean()

MASE       WMAPE       SMAPE       MAPE       MAE      RMSE
Alpha Beta                                                                 
0.05  0.05  2.165461  195.868098  103.390094  95.717817  6.142831  7.261997
      0.10  1.969928  172.899421  101.510154  91.558191  5.925228  7.060395
      0.15  1.818095  156.057019  100.180941  88.506493  5.786296  6.947515
      0.20  1.717381  145.322028   99.409453  87.044290  5.694628  6.873050
      0.25  1.647793  138.207704   98.999003  86.282070  5.634270  6.821880
...              ...         ...         ...        ...       ...       ...
0.90  0.70  1.008073   86.781673  106.063977  75.363541  4.378760  5.947993
      0.75  1.008570   87.176901  106.803468  76.237185  4.384221  5.956053
      0.80  1.009976   87.664714  107.694935  77.245912  4.390983  5.966033
      0.85  1.012094   88.222445  108.721164  78.339842  4.398302  5.978020
      0.90  1.014667   88.816352  109.869190  79.458432  4.405725  5.992094

[324 rows x 6 columns]

In [34]:
for mts in list_metrics:
    print('The optimum Alpha and Beta based on Mean', mts, 'is: {}'.format(df_result_metrics_all_params.groupby(['Alpha', 'Beta'])['MASE', 'WMAPE', 'SMAPE', 'MAPE', 'MAE', 'RMSE'].mean()[mts].idxmin()))

The optimum Alpha and Beta based on Mean MASE is: (0.75, 0.7)
The optimum Alpha and Beta based on Mean WMAPE is: (0.8, 0.5)
The optimum Alpha and Beta based on Mean SMAPE is: (0.3, 0.25)
The optimum Alpha and Beta based on Mean MAPE is: (0.65, 0.05)
The optimum Alpha and Beta based on Mean MAE is: (0.4, 0.65)
The optimum Alpha and Beta based on Mean RMSE is: (0.45, 0.6)


# Get MEDIAN metrics value of each Alpha and Beta

In [35]:
df_result_metrics_all_params.groupby(['Alpha', 'Beta'])['MASE', 'WMAPE', 'SMAPE', 'MAPE', 'MAE', 'RMSE'].median()

MASE     WMAPE     SMAPE      MAPE       MAE      RMSE
Alpha Beta                                                            
0.05  0.05  1.021092  77.82690  91.32695  60.98165  3.170066  3.648742
      0.10  1.041015  77.14015  88.22745  61.32950  2.911238  3.662611
      0.15  0.985106  78.77060  87.64940  64.14850  2.862644  3.557369
      0.20  0.991340  78.20130  86.36725  64.83035  2.731829  3.499809
      0.25  0.985817  79.47405  84.41240  65.58315  2.696497  3.505359
...              ...       ...       ...       ...       ...       ...
0.90  0.70  0.918555  80.17505  88.26965  71.47595  2.325708  3.325481
      0.75  0.924962  79.75955  89.94725  71.76240  2.359633  3.321699
      0.80  0.932631  79.75955  92.36430  73.55435  2.401181  3.318443
      0.85  0.934233  79.75955  95.10375  74.13270  2.443241  3.316009
      0.90  0.934347  79.75955  96.67785  74.76415  2.485246  3.314322

[324 rows x 6 columns]

In [36]:
for mts in list_metrics:
    print('The optimum Alpha and Beta based on Median of', mts, 'is: {}'.format(df_result_metrics_all_params.groupby(['Alpha', 'Beta'])['MASE', 'WMAPE', 'SMAPE', 'MAPE', 'MAE', 'RMSE'].median()[mts].idxmin()))

The optimum Alpha and Beta based on Median of MASE is: (0.4, 0.15)
The optimum Alpha and Beta based on Median of WMAPE is: (0.45, 0.05)
The optimum Alpha and Beta based on Median of SMAPE is: (0.3, 0.25)
The optimum Alpha and Beta based on Median of MAPE is: (0.2, 0.05)
The optimum Alpha and Beta based on Median of MAE is: (0.4, 0.35)
The optimum Alpha and Beta based on Median of RMSE is: (0.35, 0.15)


# Run Croston TSB model after decide best paramaters

In [14]:
best_alpha = 0.65
best_beta = 0.05

In [15]:
start = timer()

df_result_best_params = pd.DataFrame()
for product in list_pattern_py:
    train_product = train_data[product]
    test_product = test_data[product]
    predictions = list()
    history = [x for x in train_product[num_train_needed*-1:]]
    for t in test_product.index.tolist():
        yhat = Croston_TSB(history, extra_periods=1, alpha=best_alpha, beta = best_beta)['Forecast'].iloc[-1]
        predictions.append(yhat)
        history.append(test_product[t])
    df_result_temp = pd.DataFrame({'Product': [product for count in range(len(test_product))],
                                    'Actual Data': test_product,
                                    'Forecast': predictions})
    df_result_best_params = df_result_best_params.append(df_result_temp, ignore_index=True)
    
end = timer()
print('This line of code took {} minutes'.format((end-start) / 60))

This line of code took 0.006274826666666655 minutes


In [16]:
df_result_best_params

,Product,Actual Data,Forecast
0,HOBBIES_1_066_CA_1,0.0,0.744225
1,HOBBIES_1_066_CA_1,3.0,0.707014
2,HOBBIES_1_131_CA_1,20.0,4.536914
3,HOBBIES_1_131_CA_1,8.0,12.242545
4,HOBBIES_1_140_CA_1,0.0,1.226169
...,...,...,...
457,HOBBIES_2_147_CA_4,0.0,0.000000
458,HOBBIES_2_148_CA_4,0.0,0.302293
459,HOBBIES_2_148_CA_4,0.0,0.287178
460,HOBBIES_2_149_CA_4,1.0,0.212380


In [17]:
df_result_final = df_result_best_params.groupby('Product').apply(get_metrics_result_all_params).reset_index()

In [18]:
df_result_final

,Product,MASE,WMAPE,SMAPE,MAPE,MAE,RMSE
0,HOBBIES_1_002_CA_2,0.611028,122.2056,151.1163,67.6516,1.222056,1.229055
1,HOBBIES_1_017_CA_4,0.571948,114.3897,138.4856,55.5807,1.143897,1.144352
2,HOBBIES_1_027_CA_4,NaN,NaN,NaN,NaN,0.000000,0.000000
3,HOBBIES_1_042_CA_2,inf,36.5599,44.8662,36.5599,0.731199,0.735014
4,HOBBIES_1_053_CA_3,inf,inf,200.0000,NaN,0.527902,0.528076
...,...,...,...,...,...,...,...
226,HOBBIES_2_148_CA_4,inf,inf,200.0000,NaN,0.294735,0.294832
227,HOBBIES_2_149_CA_1,1.267482,84.4988,138.1044,80.4730,1.267482,1.395354
228,HOBBIES_2_149_CA_2,inf,inf,200.0000,NaN,0.961020,0.961336
229,HOBBIES_2_149_CA_3,0.505573,101.1145,165.0497,78.8244,0.505573,0.579229


In [19]:
df_result_final.isnull().sum()

Product     0
MASE       35
WMAPE      35
SMAPE      35
MAPE       89
MAE         0
RMSE        0
dtype: int64

In [20]:
df_result_final.MAPE.mean()

62.14558802816902

In [21]:
df_result_final.to_csv('Croston_TSB_Lumpy_Metrics.csv')